https://archive.ics.uci.edu/dataset/462/drug+review+dataset+drugs+com

# Flatiron Phase 5 Project

## Aaron Galbraith

https://www.linkedin.com/in/aarongalbraith \
https://github.com/aarongalbraith

### Submitted: November 21, 2023

## working contents

- **[rough overview](#rough-overview)<br>**
- **[duplicates](#duplicates)<br>**
- **[missing values](#missing-values)<br>**
- **[contractions](#contractions)<br>**
- **[dates](#dates)<br>**
- **[ratings](#ratings)<br>**
- **[focusing on birth control](#focusing-on-birth-control)<br>**
- **[feature engineering ideas](#feature-engineering-ideas)<br>**
- **[rudimentary word cloud maker](#rudimentary-word-cloud-maker)<br>**
- **[end](#end)<br>**


## Contents

- **[Business Understanding](#Business-Understanding)<br>**
- **[Data Understanding](#Data-Understanding)**<br>
- **[Data Preparation](#Data-Preparation)**<br>
- **[Exploration](#Exploration)**<br>
- **[Modeling](#Modeling)**<br>
- **[Evaluation](#Evaluation)**<br>
- **[Recommendations](#Recommendations)<br>**
- **[Further Inquiry](#Further-Inquiry)**<br>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import FreqDist
from nltk.corpus import stopwords
import string
from wordcloud import WordCloud

import html
import contractions

import re

from IPython.display import display

In [ ]:
d1 = pd.read_csv('../data/drugsComTrain_raw.tsv', delimiter='\t', encoding='latin-1')
d2 = pd.read_csv('../data/drugsComTest_raw.tsv', delimiter='\t', encoding='latin-1')
df = pd.concat([d1,d2]).reset_index().drop(columns=['Unnamed: 0', 'index'])

# rough overview

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.drugName.value_counts()

In [ ]:
df.condition.value_counts()

In [ ]:
df.rating.value_counts()

In [ ]:
df.groupby('drugName').condition.nunique().value_counts()

This means that, for example, 2047 drugs treat one condition only, and 753 drugs treat two conditions, etc.

In [ ]:
df.groupby('condition').drugName.nunique().value_counts()

This means that 180 conditions are treatable by two drugs, etc.

In [ ]:
pd.set_option("display.max_rows", None)
print(df.drugName.value_counts())
pd.set_option("display.max_rows", 10)

A casual overview of the drug names indicates that they all seem valid.

In [ ]:
pd.set_option("display.max_rows", None)
print(df.condition.value_counts())
pd.set_option("display.max_rows", 10)

Oddly, the condition labels often (always?) omit initial 'F' and terminal 'r'. We can isolate instances of the former by searching for conditions that start with a lower case letter.

In [ ]:
def condition_fix(condition):
    if condition.split()[-1] in ['Disorde', 'eve', 'Shoulde']:
        condition = condition+'r'
    if condition.split()[0] in ['acial', 'ibrocystic', 'ungal', 'amilial', 'ailure', 'ever', \
                                'emale', 'unctional', 'actor', 'ibromyalgia', 'atigue']:
        condition = 'F'+condition
    if condition.split()[0] in ['llicular', 'llicle', 'lic']:
        condition = 'Fo'+condition
    if condition.split()[0] in ['mance']:
        condition = 'Perfor'+condition
    return condition

In [ ]:
df.condition = df.condition.apply(lambda x: condition_fix(x))

In [ ]:
bad_conditions = []
for condition in set(df.condition):
    if ord(condition[0]) in range(97,123) and condition != 'missing':
        bad_conditions.append(condition)

In [ ]:
bad_conditions

In [ ]:
df[df.condition.isin(bad_conditions)][['drugName', 'condition']].value_counts()

# duplicates

In [ ]:
df.duplicated().value_counts()

In [ ]:
df[df.duplicated()]

In [ ]:
df[df.review == df.review.loc[178703]]

This is curious. The same review is recorded four times. There are two identical pairs, where the difference between the pairs is the drug name. We can drop one from each pair, but this will need to be revisited.

In [ ]:
df.drop_duplicates(inplace=True)

The main thing we should be aware of is records with duplicate reviews. As long as the review is unique, the record is most likely genuine.

In [ ]:
df.duplicated(subset=['review']).value_counts()

An enormous number of records have duplicated reviews.

In [ ]:
df.duplicated(subset=df.columns.difference(['drugName'])).value_counts()

Most of the duplicate reviews are accounted for by different drug names.

In [ ]:
df[df.duplicated(subset=['review'])].head()

In [ ]:
def show_dupes(index):
    print(df.review.loc[index])
    display(df[df.review == df[df.duplicated(subset=['review'])].loc[index].review][['drugName', 'condition', 'rating', 'date', 'usefulCount']])

In [ ]:
show_dupes(524)

In [ ]:
show_dupes(574)

In [ ]:
show_dupes(726)

In [ ]:
show_dupes(1070)

In [ ]:
show_dupes(1375)

In all three instances we checked, the duplicated record occurs because it is listed once under its chemical name and once under its brand name.

In [ ]:
len(df[df.duplicated(subset=df.columns.difference(['review']))])

Records that differ only by review are possibly understandable. Identical rating and useful count seems plausible, but that they also appear on the same date seems suspicious.

In [ ]:
len(df[(df.duplicated(subset=['review'])) &
   ~df.duplicated(subset=df.columns.difference(['drugName']))
  ])

This is how many records have identical reviews but differences other than the drug name.

In [ ]:
df[(df.duplicated(subset=['review'])) &
   ~df.duplicated(subset=df.columns.difference(['drugName']))
  ].head()

In [ ]:
show_dupes(2664)

In [ ]:
show_dupes(5155)

In [ ]:
show_dupes(6465)

Some of these are just common, short reviews, e.g. "Great". But others seem to have issues with the condition label as well.

In [ ]:
df[df.condition.str[:3] == 'min'][['drugName', 'condition']].value_counts()

There appear to be many condition labels that are just truncated duplicates of their drug name labels. These should be regarded as missing values.

In [ ]:
df[df.condition.str[-1] == ')'][['drugName', 'condition']].value_counts()

Is there a way to do a search for conditions whose last "word" is a string that appears in the drug name value?

In [ ]:
len(df[df.duplicated(subset=df.columns.difference(['condition']))])

In [ ]:
len(df[df.duplicated(subset=df.columns.difference(['rating']))])

In [ ]:
len(df[df.duplicated(subset=df.columns.difference(['date']))])

In [ ]:
len(df[df.duplicated(subset=df.columns.difference(['usefulCount']))])

In [ ]:
df[df.duplicated(subset=df.columns.difference(['usefulCount']))].head()

In [ ]:
show_dupes(42728)

In [ ]:
show_dupes(61617)

In [ ]:
show_dupes(69518)

In [ ]:
show_dupes(72794)

This appears to be an instance of someone re-posting a review multiple times. It seems that we should drop the duplicates in this case, but possibly we should tally up the useful count?

# missing values

In [ ]:
len(df[df.condition.isna()])

In [ ]:
df.condition.fillna('missing', inplace=True)

In [ ]:
len(df[df.condition == 'missing'])

In [ ]:
def missing_fix(string):
    if 'users found this comment' in string:
        return True
    elif 'Not Listed' in string:
        return True
    else:
        return False

df['condition'] = df.condition.apply(lambda x: 'missing' if missing_fix(x) else x)

In [ ]:
len(df[df.condition == 'missing'])

There are around 3000 records with missing conditions. For every record with a missing condition, we will assign it the condition that is most common for the drug indicated by that record.

In [ ]:
drugs_w_missing_condition = list(set(df[df.condition == 'missing'].drugName))

In [ ]:
len(drugs_w_missing_condition)

This applies to about a quarter of the drugs. We'll create a dictionary that reports the most common condition for these drugs.

In [ ]:
most_common_condition = {}

for drug in drugs_w_missing_condition:
    condition = df[df.drugName == drug].condition.value_counts().idxmax()
    if condition == 'missing' and len(set(df[df.drugName == drug].condition)) > 1:
        condition = df[(df.drugName == drug) &
                       (df.condition != 'missing')
                      ].condition.value_counts().idxmax()
    proportion = round(df[df.drugName == drug].condition.value_counts(normalize=True)[0],2)
    most_common_condition[drug] = [condition, proportion]

In [ ]:
most_common_condition['Viagra']

For example, if a review with an unlisted condition is about Viagra, we will assume the condition is Erectile Dysfunction.

In [ ]:
df['condition'] = df.apply(lambda x: most_common_condition[x.drugName][0] \
                           if x.condition == 'missing' \
                           else x.condition, axis = 1)

In [ ]:
len(df[df.condition == 'missing'])

This is how many records there are that still have no label for condition. This means the drugs indicated in these records are *only* indicated in references without an indicated condition. As such, there's not really anything we can do with these records.

In [ ]:
df.condition.value_counts()

# contractions

Here is an example of a contraction.

In [ ]:
df.review[3][56:69]

Here is how the html function fixes it.

In [ ]:
html.unescape(df.loc[3][2])[56:64]

Here is how the contractions function fixes (the html function's fix of) it.

In [ ]:
contractions.fix(html.unescape(df.loc[3][2]))[56:65]

Here is an instance of "ain't" with the same functions applied.

In [ ]:
df.review.loc[507][75:99]

In [ ]:
html.unescape(df.review.loc[507])[75:94]

In [ ]:
contractions.fix(html.unescape(df.review.loc[507]))[75:96]

In [ ]:
len(df[df.review.str.contains('ain&#039;t')])

There are 53 instances of "ain't".

I'm currently having difficulty downloading the package that appropriately fixes "ain't" into "is not" or "are not" etc. This shouldn't matter after I remove stop words. I think it will be helpful to exclude negatives like "no" and "not" from the stop words. It could certainly be of help to look for bigrams like "not good".

In [ ]:
df.review = df.review.apply(lambda x: html.unescape(x))

# dates

In [ ]:
sample = df.date.loc[0]

In [ ]:
sample

In [ ]:
re.split(r'\W+', sample)

There's probably a datetime method for this, but the following will produce month // day // year, and then we can figure out the earliest and latest dates.

In [ ]:
df['month'] = df.date.apply(lambda x: re.split(r'\W+', x)[0])
df['day'] = df.date.apply(lambda x: int(re.split(r'\W+', x)[1]))
df['year'] = df.date.apply(lambda x: int(re.split(r'\W+', x)[2]))

In [ ]:
df.year.min()

In [ ]:
df[df.year == 2008].month.value_counts()

In [ ]:
df[(df.year == 2008) &
   (df.month == 'February')
  ].day.min()

In [ ]:
df.year.max()

In [ ]:
df[df.year == 2017].month.value_counts()

In [ ]:
df[(df.year == 2017) &
   (df.month == 'November')
  ].day.max()

The reviews span from February 24, 2008 to November 30, 2017.

# ratings

In [ ]:
len(df)/2

In [ ]:
df.rating.value_counts()

In [ ]:
len(df[df.rating > 8.5])

In [ ]:
len(df[df.rating < 8.5])

To split the review roughly in half we would split between 8 and 9

To split the ratings roughly in half we would make the splits 1-8 and 9-10.

In [ ]:
len(df)/3

In [ ]:
len(df[df.rating > 9.5])

In [ ]:
len(df[df.rating < 6.5])

To split the ratings roughly in thirds we would make the splits 1-6, 7-9, and 10.

# focusing on birth control

In [ ]:
len(df[df.condition == 'Birth Control'])

This many records pertain to the condition of birth control.

In [ ]:
birth_control_drugs = set(df[df.condition == 'Birth Control'].drugName)

In [ ]:
len(birth_control_drugs)

This many drugs treat birth control.

In [ ]:
list(set(df[(df.condition != 'Birth Control') &
   (df.drugName.isin(birth_control_drugs))
  ].condition))

In [ ]:
df[df.condition == 'Birth Control'].drugName.value_counts()

# feature engineering ideas

- word count
- character count
- words in all caps
- average word length
- whether words are in English (spelled correctly)

# rudimentary word cloud maker

In [ ]:
# make list of all reviews
reviews_pos = dfbcpos.review.to_list()
reviews_neg = dfbcneg.review.to_list()

In [ ]:
# # make tokenizer
# tokenizer = TweetTokenizer(
#     preserve_case=False,
#     strip_handles=True
# )

# create list of tokens from data set
tokens_pos = word_tokenize(','.join(reviews_pos))
tokens_neg = word_tokenize(','.join(reviews_neg))


# tokens = [word for word in tokens]

In [ ]:
# make lemmatizer
lemmatizer = WordNetLemmatizer()

# lemmatize the list of words
tokens_lemmatized_pos = [lemmatizer.lemmatize(word) for word in tokens_pos]
tokens_lemmatized_neg = [lemmatizer.lemmatize(word) for word in tokens_neg]

In [ ]:
# show the most frequently occurring tokens
FreqDist(tokens_lemmatized_pos).most_common(25)

In [ ]:
# show the most frequently occurring tokens
FreqDist(tokens_lemmatized_neg).most_common(25)

In [ ]:
# obtain the standard list of stopwords
nltk.download('stopwords', quiet=True)
# start our own list of stopwords with these words
stop_list = stopwords.words('english')
# add punctuation characters
for char in string.punctuation:
    stop_list.append(char)
# add empty string
stop_list.extend(['', 'ha', 'wa'])

In [ ]:
# make stopped list of tokens
tokens_stopped_pos = [word for word in tokens_lemmatized_pos if word not in stop_list]
tokens_stopped_neg = [word for word in tokens_lemmatized_neg if word not in stop_list]

In [ ]:
# show the most frequently occurring tokens
FreqDist(tokens_stopped_pos).most_common(25)

In [ ]:
# show the most frequently occurring tokens
FreqDist(tokens_stopped_neg).most_common(25)

In [ ]:
# a function that generates a word cloud of a given list of words
def make_wordcloud(wordlist, colormap='Greens', title=None):
    # instantiate wordcloud
    wordcloud = WordCloud(
        width=600,
        height=400,
        colormap=colormap,
        collocations = True
    )
    return wordcloud.generate(','.join(wordlist))

def plot_wordcloud(wordcloud):
    # plot wordcloud
    plt.figure(figsize = (12, 15)) 
    plt.imshow(wordcloud) 
    plt.axis('off');

In [ ]:
# word cloud of stopped words
plot_wordcloud(make_wordcloud(tokens_stopped_pos))

In [ ]:
# word cloud of stopped words
plot_wordcloud(make_wordcloud(tokens_stopped_neg))

# end